In [13]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# read in business data
data = pd.read_csv("./new_business_data.csv", dtype={"abn": "str"})

# clean data

# use a headless browser (saves time)
chrome_options = Options()
# chrome_options.add_argument("--headless")

# set up the browser
driver = webdriver.Chrome(options=chrome_options)

# begin loop
driver.get("https://www.tenders.gov.au/cn/search")
abn_field = WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, "form-SupplierAbn"))
)

abn_field.clear()
abn_field.send_keys(data.iloc[0]["abn"])
driver.find_element_by_css_selector("#actions .btn-search").click()